In [ ]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
from datetime import timedelta
import time
import matplotlib.pyplot as plt
import csv
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import *
from moviepy.editor import VideoFileClip
import ffmpeg

## Step 1: Global inputs

In [ ]:
videoRecreate = False # set to True, regenerate video when source video cannot be displayed

shotsAfter = 0 # number of shots to add after the end shot with recognized faces
interval = 30 # inverval in seconds, it determines the granuity of the clips

# Inputs are needed below
videosDir = ""
videoExt = ".mp4"
characterName = ""
outputExt = ".mp4"
opSeconds = 0 # opening song in seconds
edSeconds = 0 # ending song in seconds
interval = 5 # inverval in seconds, it determines the granuity of the clips

# create the directories to store intermediate and final results
knownFacesDir = videosDir + "/knownFaces"
attendanceDir = videosDir + "/attendance"
testOutputDir = videosDir + "/test" # dir to save images of recognized faces
statsDir = videosDir + "/scenes"
timeRangesDir = videosDir + "/time_ranges"
outputDir = videosDir + "/" + characterName

##  Step 2: Scene detect for series

In [ ]:
for filename in os.listdir(videosDir):
    if filename.endswith(videoExt):
        video_name = os.path.splitext(filename)[0]
        video_file = os.path.join(videosDir, filename)
        stats_file = os.path.join(statsDir, video_name + "_stats.csv")
        !scenedetect -i "$video_file" list-scenes -s -f "$stats_file" detect-adaptive

## Step 3: Detect Faces and Mark Attendance 

In [ ]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

# TODO: allow multiple character names
def recognizeFaces(inputFile, attendanceDir, testOutputDir, \
                   classNames, characterName, encodeListKnown):
    
    cap = cv2.VideoCapture(inputFile)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        exit()
    
    # Use os.path.basename to extract the filename without the path
    filename = os.path.basename(inputFile)
    filename = filename.replace('/', '')
    videoName = os.path.splitext(filename)[0]
    attendanceFile = attendanceDir + "/" + videoName + ".csv"
    testOutput = testOutputDir + "/" + videoName
    
    if not os.path.exists(testOutput):
        os.makedirs(testOutput)

    frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    original_fps = int(cap.get(cv2.CAP_PROP_FPS))  # Original frames per second
    new_fps = 1      # Desired new frames per second

    if videoRecreate:
        # file extension hardcode here, only .mp4 is supported at this moment
        output_video_path = os.path.splitext(inputFile)[0] + ".mp4"
        # Define the codec and create VideoWriter object for the output video
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # hardcode here, Codec for the output video (MP4V for MP4)
        out = cv2.VideoWriter(output_video_path, fourcc, original_fps, frame_size)
    
    frame_skip_factor = original_fps / new_fps
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_counter = -1
    while True:
        frame_counter += 1
        # read frames

        ret, img = cap.read()
        # check if frame is None
        if img is None:
            #if True break the infinite loop
            break
            
        if videoRecreate:
            out.write(img)
            
        if frame_counter % frame_skip_factor != 0:
             continue

        imgS = cv2.resize(img,(0,0),None,0.25,0.25)
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

        facesCurFrame = face_recognition.face_locations(imgS)
        encodesCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)

        seconds = frame_counter / frame_skip_factor

        for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)
            #print(faceDis)
            matchIndex = np.argmin(faceDis)

            # TODO: fine-tune threshold
            if faceDis[matchIndex]< 0.4:
                name = classNames[matchIndex].upper()
                if characterName in name:
                    markAttendance(name, seconds, attendanceFile)
                    if not os.path.exists(testOutputDir):
                        os.makedirs(testOutputDir)
        
                    y1, x2, y2, x1 = faceLoc
                    y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
                    cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
                    cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
                    cv2.putText(img,f'{name} {round(faceDis[matchIndex],2)}',(x1+6,y2-6),\
                                cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                    cv2.imwrite( testOutput + "/" + \
                                str(timedelta(seconds=seconds)).replace(":", "m") + '.jpg', img)
                    break

            else:
                name = 'Unknown'
    
    cap.release()
    #out.release()
    
    

    cv2.destroyAllWindows()

def markAttendance(name, seconds, attendanceFile):
    # Define the header line
    header = "Name,Time In"

    with open(attendanceFile, 'a+') as f:
        # Check if the file is empty (no previous data)
        f.seek(0, 2)  # Go to the end of the file
        if f.tell() == 0:
            # If the file is empty, write the header
            f.write(header + '\n')

        time_in = timedelta(seconds=seconds)

        # Convert timedelta objects to strings
        time_in_str = str(time_in)

        # Append the attendance record
        f.write(f'{name},{time_in_str}\n')
        
def main():
    # Check if the folder exists; if not, create it
    if not os.path.exists(testOutputDir):
        os.makedirs(testOutputDir)
    if not os.path.exists(attendanceDir):
        os.makedirs(attendanceDir)
    
    print(type(characterName), characterName)
    
    images = []
    classNames = []
    myList = os.listdir(knownFacesDir)
    for cl in myList:
        if cl.startswith("."):
            continue
        curImg = cv2.imread(f'{knownFacesDir}/{cl}')
        images.append(curImg)
        classNames.append(os.path.splitext(cl)[0])

    encodeListKnown = findEncodings(images)
    
    i = 1
    for filename in os.listdir(videosDir):
        if filename.endswith(videoExt):
            recognizeFaces(videosDir + "/" + filename, attendanceDir, testOutputDir, \
                           classNames, characterName, encodeListKnown)
            # TODO: handle error if next functions depend on the success run of recognizing Faces
            print("#" + str(i) + " processed")
            i = i + 1

In [ ]:
if __name__ == "__main__":
    main()

## Step 4: Combine the results of facial recognition and scene detect

In [ ]:
def read_time_spots(inputFile):
    # Define the time spots as a list of time strings
    file = open(inputFile, 'r') 
    csv1 = csv.reader(file,delimiter=',')
    next(csv1, None)
    time_spots = []

    for row in csv1:
        time_spots.append(row[1] + ".000")
    time_spots = [datetime.strptime(time_spot, "%H:%M:%S.%f") for time_spot in time_spots]
    
    return time_spots
    
def read_scene_time_ranges(inputFile):
    
    # Initialize the list to store time ranges
    time_ranges = []

    # Read the CSV file
    with open(inputFile, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        # Iterate through the CSV rows
        for row in csv_reader:
            start_time = timedelta(seconds = float(row['Start Time (seconds)']))
            start_time_str = str(start_time)
            end_time = timedelta(seconds = float(row['End Time (seconds)']))
            end_time_str = str(end_time)
            if '.' not in start_time_str:
                start_time_str += ".000"
            if '.' not in end_time_str:
                end_time_str += ".000"
            time_ranges.append((start_time_str, end_time_str))
            
    # Convert time strings to datetime objects for easy comparison
    time_ranges = [(datetime.strptime(str(start), "%H:%M:%S.%f"), datetime.strptime(str(end), "%H:%M:%S.%f")) for start, end in time_ranges]
    
    return time_ranges

def combine_time_ranges(time_ranges):
    if not time_ranges:
        return []
    
    # TODO: fine tune interval
    allow_interval_sec = timedelta(seconds = interval)

    sorted_ranges = sorted(time_ranges)
    combined_ranges = [sorted_ranges[0]]

    for start, end in sorted_ranges[1:]:
        last_start, last_end = combined_ranges[-1]

        if start <= last_end + allow_interval_sec:
            combined_ranges[-1] = (last_start, max(last_end, end))
        else:
            combined_ranges.append((start, end))

    return combined_ranges

def create_unique_time_ranges(time_spots, time_ranges, outputFile):
    # Create a set to store the unique time ranges
    unique_time_ranges = set()

    # Iterate through time spots and check if they fall within any time range
    for spot in time_spots:
        for i in range(len(time_ranges)):
            start, end = time_ranges[i]
            if start <= spot <= end:
                
                if i + shotsAfter < len(time_ranges):
                    end = time_ranges[i + shotsAfter][1]
                    
                unique_time_ranges.add((start, end))
                
                break  # No need to check other time ranges once a match is found

    # Sort and combine overlapping time ranges
    sorted_time_ranges = combine_time_ranges(unique_time_ranges)

    # Save the time ranges to the CSV file
    with open(outputFile, mode="w", newline="") as csv_file:
        fieldnames = ["start_time", "end_time"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        # Write the header row
        writer.writeheader()

        # Write each time range as a row in the CSV file
        for start, end in sorted_time_ranges:
            writer.writerow({"start_time": start.strftime("%H:%M:%S.%f"), "end_time": end.strftime("%H:%M:%S.%f")})

        print(f"Time ranges have been saved to {outputFile}")
        
def main():
    if not os.path.exists(timeRangesDir):
        os.makedirs(timeRangesDir)
        
    # use videosDir here, with assumption that the .csv file names starts with videoName
    for filename in os.listdir(videosDir):
        if filename.endswith(videoExt):
            filename = os.path.basename(filename)
            filename = filename.replace('/', '')
            videoName = os.path.splitext(filename)[0]
            
            time_spots_file = attendanceDir + "/" + videoName + ".csv"
            time_spots = read_time_spots(time_spots_file)
            
            time_ranges_file = statsDir + "/" + videoName + "_stats.csv"
            time_ranges = read_scene_time_ranges(time_ranges_file)
            
            outputFile = timeRangesDir + "/" + videoName + "_time_ranges.csv"
            create_unique_time_ranges(time_spots, time_ranges, outputFile)
    
if __name__ == "__main__":
    main()

## Step 5: Get video clips by time range

In [ ]:
def get_sec(time_str):
    """Get Seconds from time."""
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + float(s)

def get_video_duration(video_path):
    try:
        video = VideoFileClip(video_path)
        duration = video.duration
        return duration
    except Exception as e:
        print(f"Error: {e}")
        return None


def main():
    if not os.path.exists(outputDir):
        os.makedirs(outputDir)
    
    for filename in os.listdir(videosDir):
        if filename.endswith(videoExt):
            filePath = videosDir + "/" + filename
            
            filename = os.path.basename(filename)
            filename = filename.replace('/', '')
            videoName = os.path.splitext(filename)[0]
            
            videoDuration = get_video_duration(filePath)
            
            opTime = opSeconds
            edTime = videoDuration - edSeconds
            
            time_ranges_file = timeRangesDir + "/" + videoName + "_time_ranges.csv"
            sample = open(time_ranges_file, 'r') 
            csv1 = csv.reader(sample,delimiter=',')
            next(csv1, None)
            n = 1
            
            # Create a list to store video clips
            video_clips = []
            
            for row in csv1:
                start = get_sec(row[0])
                end = get_sec(row[1])
                if start >= edTime or end <= opTime:
                    continue
                
                if start < edTime and end > opTime:
                    start = max(start, opTime)
                    end = min(end, edTime)
                    
                out_file = outputDir + "/" + videoName + '_' + str(n) + outputExt
                print(filePath, start, end, out_file)
                ffmpeg_extract_subclip(filePath, start, end, targetname=out_file)
                n = n + 1

if __name__ == "__main__":
    main()